In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import time
pd.options.display.float_format = '{:.5f}'.format

# User_Based_Recommendation Model 적용

## 데이터 준비 _ naver 뉴스 데이터 + 뉴스 내 댓글 데이터 총 129971개 크롤링

In [3]:
# Data load
comment = pd.read_csv("/content/drive/MyDrive/newport/Topic_Recommender/culture_comments.tsv",sep = '\t',names=['url', 'commentNo', 'user_id', 'user_name', 'comment', 'date_upload'])
news = pd.read_csv("/content/drive/MyDrive/newport/Topic_Recommender/naver_news.csv")
news.dropna(subset='text',inplace=True)

# Data merge
result = pd.merge(news,comment,on='url')
result

<ipython-input-3-1f2a3535c76d>:2: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  comment = pd.read_csv("/content/drive/MyDrive/newport/Topic_Recommender/culture_comments.tsv",sep = '\t',names=['url', 'commentNo', 'user_id', 'user_name', 'comment', 'date_upload'])


,platform,category1,category2,title,press,writer,date_upload_x,date_fix,url,sticker,text,commentNo,user_id,user_name,comment,date_upload_y
0,네이버,생활/문화,자동차/시승기,"기아, 발 막 다시 디뎠는데..中 전기차 시장, 난공불락 됐나",이데일리,이다원,2023-09-18 17:00,NaN,https://n.news.naver.com/mnews/article/018/000...,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",[이데일리 이다원 기자] 고속 성장 중인 중국 전기차 시장의 ‘자국 중심주의’가 확...,803524121894846780,3iObf,line****,우리나라같이 자국민과 자국 기업 무시하는 나라도 없을거다.,2023-09-18 17:15
1,네이버,생활/문화,자동차/시승기,"기아, 발 막 다시 디뎠는데..中 전기차 시장, 난공불락 됐나",이데일리,이다원,2023-09-18 17:00,NaN,https://n.news.naver.com/mnews/article/018/000...,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",[이데일리 이다원 기자] 고속 성장 중인 중국 전기차 시장의 ‘자국 중심주의’가 확...,803526289997693131,2Lhuu,kske****,우리나라 현기차의 행태에 개탄한다. 전기차에 중요한 밧데리 개발 협업을 그렇게 등한...,2023-09-18 17:49
2,네이버,생활/문화,자동차/시승기,"기아, 발 막 다시 디뎠는데..中 전기차 시장, 난공불락 됐나",이데일리,이다원,2023-09-18 17:00,NaN,https://n.news.naver.com/mnews/article/018/000...,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",[이데일리 이다원 기자] 고속 성장 중인 중국 전기차 시장의 ‘자국 중심주의’가 확...,803525643168907431,9J7Fu,sssj****,중국에서 기아차를 몆대팔겠다고 그러냐 ㅋ 중국은 지들끼리 놀라고하고 다른데 알아...,2023-09-18 17:39
3,네이버,생활/문화,자동차/시승기,"기아, 발 막 다시 디뎠는데..中 전기차 시장, 난공불락 됐나",이데일리,이다원,2023-09-18 17:00,NaN,https://n.news.naver.com/mnews/article/018/000...,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",[이데일리 이다원 기자] 고속 성장 중인 중국 전기차 시장의 ‘자국 중심주의’가 확...,803524947535200527,1FdVg,nasa****,짜장은 논외로 하자.....지들 끼리 하라고 하면 될듯....열라 만들어서 공터에 ...,2023-09-18 17:28
4,네이버,생활/문화,자동차/시승기,"기아, 발 막 다시 디뎠는데..中 전기차 시장, 난공불락 됐나",이데일리,이다원,2023-09-18 17:00,NaN,https://n.news.naver.com/mnews/article/018/000...,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",[이데일리 이다원 기자] 고속 성장 중인 중국 전기차 시장의 ‘자국 중심주의’가 확...,803524035441852617,2nPE4,lsj6****,중국버려,2023-09-18 17:14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,네이버,생활/문화,종교,[겨자씨] 하나님을 즐거워하십시오,국민일보,NaN,2023-09-09 03:03,NaN,https://n.news.naver.com/mnews/article/005/000...,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",“주께서 생명의 길을 내게 보이시리니 주의 앞에는 충만한 기쁨이 있고 주의 오른쪽에...,802674171388625120,5V6sz,ldo7****,지금 사람들의 얼굴에는 웃음끼를 찾아보기 어렵고 근심이 가득하고 슬픈 얼굴을 하고 ...,2023-09-09 13:22
129967,네이버,생활/문화,날씨,[날씨] 전남 해안 호우주의보...남부 최대 150mm↑,YTN,NaN,2023-07-12 01:47,2023-07-12 02:01,https://n.news.naver.com/mnews/article/052/000...,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",[앵커]\n전남 지방에 빗줄기가 강해지면서 호우주의보가 내려졌습니다.\n남부 지방은...,797158368664879176,2sowx,hi******,경남도 완전 폭풍의 언덕임,2023-07-12 02:26
129968,네이버,생활/문화,날씨,[날씨] 전남 해안 호우주의보...남부 최대 150mm↑,YTN,NaN,2023-07-12 01:47,2023-07-12 02:01,https://n.news.naver.com/mnews/article/052/000...,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",[앵커]\n전남 지방에 빗줄기가 강해지면서 호우주의보가 내려졌습니다.\n남부 지방은...,797161777291853862,c2CyN,skyr****,💗💗💗💗💗💗💗,2023-07-12 03:19
129969,네이버,생활/문화,날씨,[날씨] 전남 해안 호우주의보...남부 최대 150mm↑,YTN,NaN,2023-07-12 01:47,2023-07-12 02:01,https://n.news.naver.com/mnews/article/052/000...,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",[앵커]\n전남 지방에 빗줄기가 강해지면서 호우주의보가 내려졌습니다.\n남부 지방은...,797163425485553858,tuMU,plut****,지긋지긋한 북괴 전라도인들 폭우로 한마리도 안남기고 노인부터 아이까지 싹다 죽어없어...,2023-07-12 03:44


In [7]:
# 유저 기반 추천 시 사용할 데이터프레임 준비
new_reviews=result.copy()
new_reviews=new_reviews[['user_id','url','date_upload_x']]

# 한 기사 안에 유저 별로 남긴 댓글 수를 Rating으로 판단하기 위해 count
user_cnt = new_reviews.groupby(['user_id','url'])['date_upload_x'].count().reset_index()
user_cnt = user_cnt[user_cnt['user_id']!='6GWtz'] # 이상치 제거

# count 데이터와 합쳐 최종 input 데이터 생성
df = pd.merge(new_reviews,user_cnt,on=['url','user_id'])
df.columns = ['user_id','url','timestamp','rating']

# timestamp 변환
df['timestamp']= df['timestamp'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M'))
df['timestamp']= df['timestamp'].apply(lambda x : time.mktime(x.timetuple()))

# 최종 df
df =df[['user_id','url','rating','timestamp']]
df = df.groupby(['user_id','url','rating','timestamp']).count().reset_index()
df

,user_id,url,rating,timestamp
0,1000i,https://n.news.naver.com/mnews/article/366/000...,1,1692997020.00000
1,100IJ,https://n.news.naver.com/mnews/article/023/000...,1,1689131040.00000
2,100IJ,https://n.news.naver.com/mnews/article/119/000...,1,1689574860.00000
3,100IJ,https://n.news.naver.com/mnews/article/366/000...,1,1694322060.00000
4,100N9,https://n.news.naver.com/mnews/article/087/000...,1,1691452860.00000
...,...,...,...,...
89817,zz4l,https://n.news.naver.com/mnews/article/011/000...,1,1694414460.00000
89818,zzI5,https://n.news.naver.com/mnews/article/001/001...,2,1693270260.00000
89819,zzWe,https://n.news.naver.com/mnews/article/629/000...,1,1687651260.00000
89820,zzr0,https://n.news.naver.com/mnews/article/025/000...,1,1689055200.00000


In [8]:
df['rating'].unique()

array([1, 2, 3])

## Train-test split

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, test_size = 0.30, random_state = 42)

print(X_train.shape) # (62875, 4)
print(X_test.shape) # (26947, 4)

# 피봇 테이블 생성
user_data = X_train.pivot(index = 'user_id', columns = 'url', values = 'rating').fillna(0)
user_data.head()

(62875, 4)
(26947, 4)


url,https://n.news.naver.com/mnews/article/001/0014007927?sid=103,https://n.news.naver.com/mnews/article/001/0014008555?sid=103,https://n.news.naver.com/mnews/article/001/0014008556?sid=103,https://n.news.naver.com/mnews/article/001/0014008641?sid=103,https://n.news.naver.com/mnews/article/001/0014008673?sid=103,https://n.news.naver.com/mnews/article/001/0014009832?sid=103,https://n.news.naver.com/mnews/article/001/0014009833?sid=103,https://n.news.naver.com/mnews/article/001/0014012095?sid=103,https://n.news.naver.com/mnews/article/001/0014012153?sid=103,https://n.news.naver.com/mnews/article/001/0014015400?sid=103,...,https://n.news.naver.com/mnews/article/666/0000017516?sid=103,https://n.news.naver.com/mnews/article/666/0000017883?sid=103,https://n.news.naver.com/mnews/article/666/0000018712?sid=103,https://n.news.naver.com/mnews/article/666/0000019225?sid=103,https://n.news.naver.com/mnews/article/666/0000019541?sid=103,https://n.news.naver.com/mnews/article/666/0000020028?sid=103,https://n.news.naver.com/mnews/article/666/0000020288?sid=103,https://n.news.naver.com/mnews/article/666/0000020407?sid=103,https://n.news.naver.com/mnews/article/666/0000020900?sid=103,https://n.news.naver.com/mnews/article/666/0000021168?sid=103
user_id,,,,,,,,,,,,,,,,,,,,,
1000i,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
100IJ,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
100N9,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
100eW,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
100n7,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [17]:
# train 에는 0을 test 에는 1을 적용
dummy_train = X_train.copy()
dummy_test = X_test.copy()

dummy_train['rating'] = dummy_train['rating'].apply(lambda x: 0 if x > 0 else 1)
dummy_test['rating'] = dummy_test['rating'].apply(lambda x: 1 if x > 0 else 0)

# 사용자가 등급을 매기지 않은 url 예측을 위해 1로 표시
dummy_train = dummy_train.pivot(index = 'user_id', columns = 'url', values = 'rating').fillna(1)

# 사용자가 등급을 매기지 않은 url 평가를 위해 0으로 표시됩니다
dummy_test = dummy_test.pivot(index ='user_id', columns = 'url', values = 'rating').fillna(0)

dummy_train.head()
dummy_test.head()

url,https://n.news.naver.com/mnews/article/001/0014008555?sid=103,https://n.news.naver.com/mnews/article/001/0014008556?sid=103,https://n.news.naver.com/mnews/article/001/0014008641?sid=103,https://n.news.naver.com/mnews/article/001/0014008673?sid=103,https://n.news.naver.com/mnews/article/001/0014009832?sid=103,https://n.news.naver.com/mnews/article/001/0014009833?sid=103,https://n.news.naver.com/mnews/article/001/0014012095?sid=103,https://n.news.naver.com/mnews/article/001/0014012153?sid=103,https://n.news.naver.com/mnews/article/001/0014015762?sid=103,https://n.news.naver.com/mnews/article/001/0014018026?sid=103,...,https://n.news.naver.com/mnews/article/662/0000028448?sid=103,https://n.news.naver.com/mnews/article/662/0000028456?sid=103,https://n.news.naver.com/mnews/article/662/0000028567?sid=103,https://n.news.naver.com/mnews/article/666/0000015290?sid=103,https://n.news.naver.com/mnews/article/666/0000015508?sid=103,https://n.news.naver.com/mnews/article/666/0000017492?sid=103,https://n.news.naver.com/mnews/article/666/0000019541?sid=103,https://n.news.naver.com/mnews/article/666/0000021420?sid=103,https://n.news.naver.com/mnews/article/666/0000021523?sid=103,https://n.news.naver.com/mnews/article/666/0000021605?sid=103
user_id,,,,,,,,,,,,,,,,,,,,,
100n7,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
100os,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1036z,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
104f6,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
106Da,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


## User-User Similarity matrix

In [1]:
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 유사도 행렬, 코사인 유사도 반영
user_similarity = cosine_similarity(user_data)
user_similarity[np.isnan(user_similarity)] = 0
print(user_similarity)
print(user_similarity.shape)

NameError: ignored

## Predicting the User ratings on the Url

In [ ]:
user_predicted_ratings = np.dot(user_similarity, user_data)
user_predicted_ratings.shape

In [ ]:
# np.multiply for cell-by-cell multiplication

user_final_ratings = np.multiply(user_predicted_ratings, dummy_train)
user_final_ratings.head()

## Top 5 URL recommendations for the User 42

In [ ]:
user_final_ratings.iloc[42].sort_values(ascending = False)[0:5]

In [ ]:
# 유저 별 추천된 순서로 출력되는 데이터프레임
recommender = user_final_ratings.iloc[1].sort_values(ascending = False).reset_index()
recommender_result = recommender[recommender['100IJ']>0] # 이상치 제거
recommender_result

# 토픽 모델링 결과 연결

In [ ]:
# 토픽 모델링 - 추천 결과 merge
topic = pd.read_csv("/content/drive/MyDrive/newport/Topic_Recommender/doc_topic_df.csv")
reco_topic = pd.merge(recommender_result,topic,on='url',how='left')
reco_topic

,url,100IJ,Topic,category2,title,text,press,date_upload,sticker,Percentage
0,https://n.news.naver.com/mnews/article/119/000...,9.33999,5,도로/교통,"5호선 노선 협의체, 지자체 합의 머뭇거리는 사이…“지역 갈등 커진다”",[데일리안 = 임정희 기자] 지하철 5호선 연장과 관련해 노선 협의체 운영 내용에 ...,데일리안,2023-07-24 05:16,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.73954
1,https://n.news.naver.com/mnews/article/421/000...,4.94483,5,도로/교통,"5호선 김포연장 노선, 김포시민이 절대 포기 못하는 이유는?",(김포·인천=뉴스1) 정진욱 기자 = 대도시권광역교통위원회가 서울지하철 5호선 연장...,뉴스1,2023-08-15 06:00,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.66623
2,https://n.news.naver.com/mnews/article/421/000...,2.68729,5,도로/교통,'인천안 vs 김포안' 누구 손? …대광위 지하철 5호선 연장노선 결정 임박,(인천·김포=뉴스1) 정진욱 기자 = 서울지하철 5호선 연장노선에 대한 대도시권광역...,뉴스1,2023-09-06 17:33,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.65782
3,https://n.news.naver.com/mnews/article/421/000...,1.70517,5,도로/교통,서울지하철 5호선 연장 노선안 제출 인천 vs 김포…대광위 손은?,(인천·김포=뉴스1) 정진욱 기자 = 경기도·김포시와 인천시가 서울지하철 5호선 연...,뉴스1,2023-09-02 11:13,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.80264
4,https://n.news.naver.com/mnews/article/119/000...,1.16667,5,도로/교통,"5호선 연장 노선 결정 ‘차일피일’, 대광위 중재 역할 나설까",[데일리안 = 임정희 기자] 지하철 5호선의 김포·검단 세부 노선이 지난달을 넘기지...,데일리안,2023-09-01 06:51,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.80472
...,...,...,...,...,...,...,...,...,...,...
132,https://n.news.naver.com/mnews/article/005/000...,0.19245,2,사건사고,"엑소 디오, 대기실 흡연 딱 걸려…과태료 처분 [영상]",그룹 엑소 멤버 겸 배우 디오(본명 도경수·30)가 실내 흡연으로 과태료 처분을 받...,국민일보,2023-09-07 04:45,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.87989
133,https://n.news.naver.com/mnews/article/005/000...,0.19245,2,생활/문화 일반,‘동탄과 똑닮’…몽골 울란바토르가 ‘몽탄 신도시’로 불리는 까닭은,몽골은 요즘 ‘몽탄 신도시’라고도 불린다. 수도 울란바토르의 풍경이 경기도 동탄신도...,국민일보,2023-08-20 00:06,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.85186
134,https://n.news.naver.com/mnews/article/005/000...,0.19245,2,패션/뷰티,천원짜리가 ‘10만원’ 껑충…‘나이키’ 리폼백 문제 없나,최근 스포츠 브랜드 ‘나이키’에서 판매하는 쇼핑백을 구매해 리폼한 뒤 이를 수십 배...,국민일보,2023-08-01 00:03,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.76228
135,https://n.news.naver.com/mnews/article/421/000...,0.16667,5,도로/교통,"강범석 인천 서구청장 ""5호선 연장 인천·김포시민 모두 이동권 보장돼야""",(인천=뉴스1) 정진욱 기자 = 인천 서구는 11일 오전 아라동행정복지센터에서 ‘서...,뉴스1,2023-09-11 14:04,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.73467


In [ ]:
# 유저 선호 토픽 기반 추천 목록 출력 ('100IJ' 지수가 높은 순대로 추천을 해준다.)
input_data = input("유저가 선택한 토픽 번호를 입력해주세요")

user_result = reco_topic[reco_topic['Topic']==input_data]
user_result

,url,100IJ,Topic,category2,title,text,press,date_upload,sticker,Percentage
11,https://n.news.naver.com/mnews/article/023/000...,0.67553,3,생활/문화 일반,“보이는 즉시 사살하라!”...퓨마 출몰에 발칵 뒤집힌 미국 이 도시 [수요동물원],수요동물원 뉴스레터 구독하기 ☞ https://page.stibee.com/subs...,조선일보,2023-08-02 00:01,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.71265
22,https://n.news.naver.com/mnews/article/016/000...,0.47140,3,생활/문화 일반,‘괴물’ 변희봉 췌장암 투병 끝 별세…향년 81세,[헤럴드경제=김유진 기자] 원로배우 변희봉(본명 변인철)이 18일 별세했다. 향년 ...,헤럴드경제,2023-09-18 14:29,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.79158
24,https://n.news.naver.com/mnews/article/001/001...,0.40825,3,생활/문화 일반,영화계 큰 별 잠들다…'괴물'의 원로배우 변희봉 별세(종합),(서울=연합뉴스) 황재하 기자 = 드라마와 영화를 넘나들며 활약해온 원로배우 변희봉...,연합뉴스,2023-09-18 15:44,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.94000
25,https://n.news.naver.com/mnews/article/119/000...,0.40825,3,생활/문화 일반,"조민, 조국 이어 책 냈다…""상식적으로 살고자""",[데일리안 = 이지희 기자] 조국 전 법무부 장관 딸 조민 씨가 책을 출판한다. 제...,데일리안,2023-09-18 16:40,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.88010
26,https://n.news.naver.com/mnews/article/656/000...,0.40825,3,생활/문화 일반,"원로배우 변희봉, 암 투병 끝에 별세… 향년 81세",원로배우 변희봉(본명 변인철) 씨가 암 투병 끝에 18일 별세했다. 향년 81세.\...,대전일보,2023-09-18 14:41,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.79158
36,https://n.news.naver.com/mnews/article/009/000...,0.40825,3,책,"포르노 보여주고, 비아그라 먹여도 안된다고?...“성에 관심없는 너?” [생색(生色)]","[생색-12] 앙증맞은 얼굴에, 복슬복슬한 털들까지. 누구든 이들 앞에선 무장해제를...",매일경제,2023-09-17 06:50,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.92565
38,https://n.news.naver.com/mnews/article/020/000...,0.40825,3,날씨,태풍 ‘카눈’ 서울 바짝 붙어 관통 전망…강원 최대 600mm 폭우,제6호 태풍 ‘카눈’의 중심이 서울 동쪽으로 바짝 붙어 한반도를 관통할 것으로 보인...,동아일보,2023-08-09 03:01,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.92423
42,https://n.news.naver.com/mnews/article/020/000...,0.40825,3,날씨,시간당 최대 60mm 이번주 내내 비… “또 침수 불안”,폭염이 끝나고 다시 찾아온 장맛비가 9일을 시작으로 일주일 넘게 이어질 것으로 전망...,동아일보,2023-07-10 03:03,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.94176
53,https://n.news.naver.com/mnews/article/023/000...,0.33333,3,사건사고,한 달 2㎏ 설탕이 몸에서 빠진다?... 요즘 뜨는 당뇨 치료제,국내 당뇨병 환자는 최근 급속히 늘면서 526만명이나 됐다(2020년 조사 기준)....,조선일보,2023-06-22 03:04,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.72970
73,https://n.news.naver.com/mnews/article/094/000...,0.33333,3,여행/레저,"등산, 등반, 등정…뭐가 다르죠?","""이번 주말에 청계산으로 등반하러 갈래? 3시간이면 정상까지 다녀올 수 있어!""\n...",월간 산,2023-09-07 06:46,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'...",0.70419


# item-based recommender
- 위에서 추천된 상위 1개의 토픽과 비슷한 뉴스를 2-3개 정도 추가로 추천해줌으로써 건전성을 보충해준다.

In [ ]:
import pandas as pd

# 뉴스 데이터
# Data merge
news = pd.merge(news,comment,on='url')
news
# 결측치
news['text'] = news['text'].fillna(news['title'])
news

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 불용어 설정
korean_stopwords = list('./basic_stopword.txt')
tfidf = TfidfVectorizer(stop_words=korean_stopwords)

tfidf_matrix = tfidf.fit_transform(news['text'])

In [ ]:
# 코사인 유사도
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

cosine_sim = pd.DataFrame(cosine_sim, index=news.index, columns=news.index)
indices = pd.Series(news.index, index=news['title'])

In [ ]:
cosine_sim

In [ ]:
def recommender(title, n_of_recomm):
    idx = indices[title]
    sim_scores = cosine_sim[idx]
    sim_scores = sim_scores.sort_values(ascending=False)[1:n_of_recomm+1]
    return news.loc[sim_scores.index]['title']

In [ ]:
# 위의 데이터와 연결

upper_3 = user_result['title'][:3]

# 최종 결과 담을 리스트
Final_recommendation = []
for title in upper_3:
  Final_recommendation.append({title : recommender(title, 3):)

In [ ]:
Final_recommendation